embedding is nothing but a nemeric representation of our text, and we store these embeddings to our vector database

first create UPI key from Google palm makersuite and copy it.

In [ ]:
from langchain.llms import GooglePalm
api_keys = "----"
llm = GooglePalm(google_api_key = api_key, temperature = 0.7)

In [ ]:
#we can ask for any input as its a generative model
poem = llm("write a 4 line poem of my love for samosa")
print(poem)

In [ ]:
essay = llm("write email requesting refund for electronic item")
print(essay)

In [ ]:
#we can import csv file into Jupyter
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path = "file_name.csv", source_column = "column_name") #source_column = whichever columns contains questions provide that column
data = loader.load()

In [ ]:
#once csv data loaded lets create embedding
#we use instructor embedding
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS #for creating vector database
#from langchain.vectorstores import chroma

instructor_embeddings = HuggingfaceInstructEmbeddings()

vectordb= FAISS.from_documets(documents =data, embedding = instructor_embedding) #we need to specify documents here


In [ ]:
#it two sentences meaning are similar then cosine similarity close to 1
#if not similar then near to -1

In [ ]:
#once vector database created we can save it

In [ ]:
#we have to create a retriver object from vector database
retriever = vectordb.as_retriever() 
rdocs = retriever.get_relevant_documents("for how long is this course valid")
rdocs

#the work of this object is whenever we have new questions it wil create embedding of that question then we will pull similar looking vector from the vector database.
#soo, its takes input question it compares its embeddings with the embeddings which are stored in vector database and then it return embeddings

In [ ]:
#in order to solve issue we have to create prompt template
from langchain.prompts import PromptTemplate
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making.
If the answer is not found in the context, kindly state "I dont't know." Don't try to make up an answer.

CONTEXT: {context}

Question: {question}"""

PROMPT = PromptTemplate(
    template = prompt_template, input_variables = ["context", "question"]

In [ ]:
#we are imported retrievalQA class
#this is performing last step which is we have relevant document now we will form this prompt and ask it to LLM
from langchain.chains import RetrievalQA

#lets create object of this class
chain = RetrievalQA(llm = llm,
            chain_type = "stuff", #is can be stuff or mapreduce
            retriever = retriever,
            input_key = "query",
            return_source_documents = True,  #when we get answer then we also want to return the source document from that csv file which were relevant to this answer
            chain_type_kwargs={"prompt":PROMPT}
           )

In [ ]:
chain("ask question")
#here hellucination concept happens
#we want to tell model that only use csv for giving the answers and dont use any other source
#so its keep saying random things, and we need to stop this
#soo we just want to make it simple, follow csv if answer not present simply answer I Dont Know
